In [1]:
import sys
sys.path.append('../src') 
import numpy as np
import pandas as pd
import random
from typing import List, Tuple
from collections import defaultdict
import matplotlib.pyplot as plt
import time

class meax_mtsp:
    def __init__(self, distance_matrix: np.ndarray, m: int = 3,
                 pop_size: int = 10, generations: int = 30):
        
        self.distance_matrix = np.asarray(distance_matrix)
        self.m = int(m)
        self.pop_size = int(pop_size)
        self.generations = int(generations)
        self._n = self.distance_matrix.shape[0]
        # cache cho fitness: key = tuple(permutation)
        self._fitness_cache = {}

        # Các tham số điều chỉnh tốc độ/chiến lược (có thể chỉnh)
        self._pc = 0.8              # crossover probability (hard-coded)
        self._two_opt_j_span = 50   # giới hạn j-range khi thử 2-opt (độ rộng)
        self._vnd_sample_moves = 80 # số lượng move sampled per neighborhood (thay cho exhaustive)
        self._vnd_max_iters = 60    # tổng số vòng kiểm tra neighborhoods trong VND

    # ---------------- helpers ----------------
    def _perm_key(self, perm: List[int]) -> Tuple[int]:
        return tuple(perm)

    def _route_distance(self, route: List[int]) -> float:
        """Tính khoảng cách của 1 route bắt đầu/ending tại depot(0).
           Dùng numpy indexing cho tốc độ."""
        if not route:
            return 0.0
        arr = [0] + route + [0]
        prevs = np.array(arr[:-1], dtype=int)
        nexts = np.array(arr[1:], dtype=int)
        return float(self.distance_matrix[prevs, nexts].sum())

    # ---------------- Step 1: Initialization ----------------
    def initialize_population(self) -> List[List[int]]:
        cities = list(range(1, self._n))
        return [random.sample(cities, len(cities)) for _ in range(self.pop_size)]

    # ---------------- Decode and perm helpers ----------------
    def decode_solution(self, chromosome: List[int]) -> List[List[int]]:
        routes = [[] for _ in range(self.m)]
        for i, city in enumerate(chromosome):
            routes[i % self.m].append(city)
        return routes

    def perm_from_routes(self, routes: List[List[int]]) -> List[int]:
        idxs = [0] * len(routes)
        perm = []
        while True:
            added = False
            for r in range(len(routes)):
                if idxs[r] < len(routes[r]):
                    perm.append(routes[r][idxs[r]])
                    idxs[r] += 1
                    added = True
            if not added:
                break
        # append missing (safety)
        all_cities = set(range(1, self._n))
        if set(perm) != all_cities:
            for c in range(1, self._n):
                if c not in perm:
                    perm.append(c)
        return perm

    # ---------------- Step 2: Fitness with caching ----------------
    def calculate_fitness(self, chromosome: List[int]) -> float:
        key = self._perm_key(chromosome)
        v = self._fitness_cache.get(key)
        if v is not None:
            return v
        # compute max route distance (vectorized per route)
        routes = self.decode_solution(chromosome)
        maxd = 0.0
        for r in routes:
            d = self._route_distance(r)
            if d > maxd:
                maxd = d
        self._fitness_cache[key] = maxd
        return maxd

    # ---------------- mEAX crossover (kept name mEAX_crossover) ----------------
    def mEAX_crossover(self, p1: List[int], p2: List[int]) -> Tuple[List[int], List[int]]:
        if random.random() > self._pc:
            return p1.copy(), p2.copy()

        def edges_from_perm(perm):
            edges = set()
            size = len(perm)
            for i in range(size):
                a = perm[i]; b = perm[(i + 1) % size]
                edges.add((min(a, b), max(a, b)))
            return edges

        A_edges = edges_from_perm(p1)
        B_edges = edges_from_perm(p2)

        edge_label = {}
        for e in A_edges:
            edge_label.setdefault(e, set()).add('A')
        for e in B_edges:
            edge_label.setdefault(e, set()).add('B')

        adj_A = defaultdict(list)
        adj_B = defaultdict(list)
        for (u, v) in A_edges:
            adj_A[u].append(v); adj_A[v].append(u)
        for (u, v) in B_edges:
            adj_B[u].append(v); adj_B[v].append(u)

        all_edges = set(edge_label.keys())
        used_edge = set()
        AB_cycles = []

        # build AB-cycles (alternate A/B) — use random tie-breaking to avoid worstcase determinism
        while True:
            unused = [e for e in all_edges if e not in used_edge]
            if not unused:
                break
            start_edge = unused[0]
            cycle = []
            u0, v0 = start_edge
            current_node = u0
            prev_edge = start_edge
            next_label = 'B' if 'A' in edge_label[start_edge] else 'A'
            cycle.append(prev_edge)
            used_edge.add(prev_edge)
            while True:
                a, b = prev_edge
                current_node = b if current_node == a else a
                candidate_edges = []
                if next_label == 'A':
                    for nb in adj_A[current_node]:
                        edge = (min(current_node, nb), max(current_node, nb))
                        if edge not in used_edge:
                            candidate_edges.append(edge)
                else:
                    for nb in adj_B[current_node]:
                        edge = (min(current_node, nb), max(current_node, nb))
                        if edge not in used_edge:
                            candidate_edges.append(edge)
                if not candidate_edges:
                    break
                # pick random candidate to reduce bias and runtime patterns
                next_edge = random.choice(candidate_edges)
                cycle.append(next_edge)
                used_edge.add(next_edge)
                prev_edge = next_edge
                next_label = 'A' if next_label == 'B' else 'B'
                if prev_edge == start_edge:
                    break
            if cycle:
                AB_cycles.append(cycle)

        # fallback simple OX if no cycles found
        if not AB_cycles:
            size = len(p1)
            a, b = sorted(random.sample(range(size), 2))
            def ox(pA, pB):
                child = [None]*size
                child[a:b] = pA[a:b]
                pos = b
                for city in pB[b:] + pB[:b]:
                    if city not in child:
                        if pos >= size:
                            pos = 0
                        child[pos] = city
                        pos += 1
                return child
            return ox(p1, p2), ox(p2, p1)

        cycles_idx = list(range(len(AB_cycles)))
        sel1 = set(i for i in cycles_idx if random.random() < 0.5)
        if not sel1:
            sel1.add(random.choice(cycles_idx))
        sel2 = set(i for i in cycles_idx if random.random() < 0.5)
        if not sel2:
            sel2.add(random.choice(cycles_idx))

        def build_child_edges(selected_idx_set, base_edges):
            child_edges = set(base_edges)
            for i in selected_idx_set:
                for e in AB_cycles[i]:
                    if e in child_edges:
                        child_edges.remove(e)
                    else:
                        child_edges.add(e)
            return child_edges

        child1_edges = build_child_edges(sel1, A_edges)
        child2_edges = build_child_edges(sel2, A_edges)

        def edges_to_cycles(edge_set):
            adj = defaultdict(list)
            for (u, v) in edge_set:
                adj[u].append(v); adj[v].append(u)
            cycles = []
            visited = set()
            for node in adj:
                if node in visited:
                    continue
                curr = node; prev = None; seq = []
                while True:
                    seq.append(curr)
                    visited.add(curr)
                    nbrs = adj[curr]
                    if not nbrs:
                        break
                    next_node = None
                    if len(nbrs) == 1:
                        next_node = nbrs[0]
                    else:
                        next_node = nbrs[0] if nbrs[0] != prev else nbrs[1]
                    prev, curr = curr, next_node
                    if curr == node or curr is None:
                        break
                if seq:
                    cycles.append(seq)
            return cycles

        cycles1 = edges_to_cycles(child1_edges)
        cycles2 = edges_to_cycles(child2_edges)

        def stitch_cycles(cycles):
            if not cycles:
                return []
            seqs = [c.copy() for c in cycles]
            seqs.sort(key=lambda s: -len(s))
            perm = seqs[0].copy()
            remaining = seqs[1:]
            while remaining:
                best_idx = None; best_cost = float('inf'); best_orient = False
                for i, s in enumerate(remaining):
                    # compute only two seam costs
                    cost1 = self.distance_matrix[perm[-1]][s[0]]
                    cost2 = self.distance_matrix[perm[-1]][s[-1]]
                    if cost1 < best_cost:
                        best_cost = cost1; best_idx = i; best_orient = False
                    if cost2 < best_cost:
                        best_cost = cost2; best_idx = i; best_orient = True
                s = remaining.pop(best_idx)
                if best_orient:
                    s = s[::-1]
                perm.extend(s)
            # append missing
            all_cities = set(range(1, self._n))
            missing = list(all_cities - set(perm))
            if missing:
                perm.extend(missing)
            return perm

        perm1 = stitch_cycles(cycles1)
        perm2 = stitch_cycles(cycles2)

        # limited 2-opt repair (restricted j-range and iterations)
        def two_opt_local(perm, max_iters=80):
            best = perm
            best_fit = self.calculate_fitness(best)
            n = len(perm)
            it = 0
            improved = True
            while it < max_iters and improved:
                improved = False
                # sample some (i,j) pairs rather than exhaustive
                sample_pairs = min(200, n * 5)
                for _ in range(sample_pairs):
                    i = random.randrange(0, n - 2)
                    j = random.randrange(i + 2, min(n, i + 2 + self._two_opt_j_span))
                    new = best[:i+1] + best[i+1:j+1][::-1] + best[j+1:]
                    new_fit = self.calculate_fitness(new)
                    if new_fit + 1e-12 < best_fit:
                        best = new; best_fit = new_fit; improved = True; break
                it += 1
            return best

        perm1 = two_opt_local(perm1, max_iters=60)
        perm2 = two_opt_local(perm2, max_iters=60)

        # validate perms
        def validate_perm(perm):
            all_cities = set(range(1, self._n))
            if set(perm) != all_cities or len(perm) != len(all_cities):
                seen = set(); new = []
                for x in perm:
                    if x not in seen and 1 <= x < self._n:
                        new.append(x); seen.add(x)
                for x in range(1, self._n):
                    if x not in seen:
                        new.append(x); seen.add(x)
                return new
            return perm

        perm1 = validate_perm(perm1)
        perm2 = validate_perm(perm2)
        return perm1, perm2

    # ---------------- VND (lightweight with sampling & local delta) ----------------
    def vnd_improvement(self, chromosome: List[int], max_iters: int = None) -> List[int]:
        if max_iters is None:
            max_iters = self._vnd_max_iters
        current = chromosome.copy()
        current_fit = self.calculate_fitness(current)

        neighborhoods = ['2opt', 'relocate', 'swap']
        k = 0
        it = 0
        while it < max_iters and k < len(neighborhoods):
            improved = False
            routes = self.decode_solution(current)

            # 1) intra-route 2-opt (sampled)
            if neighborhoods[k] == '2opt':
                best_perm = current; best_fit = current_fit
                # sample up to S moves across all routes
                S = self._vnd_sample_moves
                for _ in range(S):
                    # select route with prob proportional to length (longer routes more likely)
                    lens = [len(r) for r in routes]
                    if sum(lens) == 0:
                        break
                    idx = random.choices(range(len(routes)), weights=[l+1 for l in lens], k=1)[0]
                    r = routes[idx]
                    lr = len(r)
                    if lr < 4:
                        continue
                    i = random.randrange(0, lr - 2)
                    j = random.randrange(i + 1, min(lr, i + 1 + self._two_opt_j_span))
                    # compute route-level delta: old_distance and new_distance
                    old_dist = self._route_distance(r)
                    new_r = r[:i+1] + r[i+1:j+1][::-1] + r[j+1:]
                    new_dist = self._route_distance(new_r)
                    if new_dist < old_dist - 1e-12:
                        # apply change
                        new_routes = [list(rr) for rr in routes]
                        new_routes[idx] = new_r
                        new_perm = self.perm_from_routes(new_routes)
                        new_fit = max(current_fit, new_dist) if old_dist < current_fit else self.calculate_fitness(new_perm)
                        # safer: compute full fitness (but only when promising) -> we compute full fitness to be accurate
                        new_fit = self.calculate_fitness(new_perm)
                        if new_fit + 1e-12 < best_fit:
                            best_fit = new_fit; best_perm = new_perm; improved = True; break
                if improved:
                    current = best_perm; current_fit = best_fit; k = 0

            # 2) relocate (sampled moves)
            elif neighborhoods[k] == 'relocate':
                best_perm = current; best_fit = current_fit
                S = self._vnd_sample_moves
                # target moves from longer routes
                r_dists = [self._route_distance(r) for r in routes]
                order_src = sorted(range(len(routes)), key=lambda x: -r_dists[x])
                tries = 0
                while tries < S:
                    if not order_src:
                        break
                    src = random.choice(order_src[:min(len(order_src), 3)])
                    if not routes[src]:
                        tries += 1; continue
                    pos = random.randrange(0, len(routes[src]))
                    city = routes[src][pos]
                    dst = random.randrange(0, len(routes))
                    if dst == src:
                        tries += 1; continue
                    insert_pos = random.randrange(0, len(routes[dst]) + 1)
                    new_routes = [list(rr) for rr in routes]
                    new_routes[src].pop(pos)
                    new_routes[dst].insert(insert_pos, city)
                    new_perm = self.perm_from_routes(new_routes)
                    new_fit = self.calculate_fitness(new_perm)
                    if new_fit + 1e-12 < best_fit:
                        best_fit = new_fit; best_perm = new_perm; improved = True; break
                    tries += 1
                if improved:
                    current = best_perm; current_fit = best_fit; k = 0

            # 3) swap (sampled)
            elif neighborhoods[k] == 'swap':
                best_perm = current; best_fit = current_fit
                S = self._vnd_sample_moves
                tries = 0
                # pick from top 3 routes by distance to focus on problematic routes
                r_dists = [self._route_distance(r) for r in routes]
                top_routes = sorted(range(len(routes)), key=lambda x: -r_dists[x])[:min(3, len(routes))]
                while tries < S:
                    if len(top_routes) < 2:
                        break
                    i_idx, j_idx = random.sample(top_routes, 2)
                    if not routes[i_idx] or not routes[j_idx]:
                        tries += 1; continue
                    a = random.randrange(0, len(routes[i_idx]))
                    b = random.randrange(0, len(routes[j_idx]))
                    new_routes = [list(rr) for rr in routes]
                    new_routes[i_idx][a], new_routes[j_idx][b] = new_routes[j_idx][b], new_routes[i_idx][a]
                    new_perm = self.perm_from_routes(new_routes)
                    new_fit = self.calculate_fitness(new_perm)
                    if new_fit + 1e-12 < best_fit:
                        best_fit = new_fit; best_perm = new_perm; improved = True; break
                    tries += 1
                if improved:
                    current = best_perm; current_fit = best_fit; k = 0

            if not improved:
                k += 1
            it += 1

        return current

    # ---------------- Step 5: Main Loop ----------------
    def run(self):
        # clear cache at start
        self._fitness_cache.clear()
        population = self.initialize_population()
        best_solution = None
        best_fitness = float('inf')
        fitness_history = []

        # pre-evaluate initial pop (and cache)
        pop_fitness = [self.calculate_fitness(ind) for ind in population]

        for gen in range(self.generations):
            # track best
            min_idx = int(np.argmin(pop_fitness))
            if pop_fitness[min_idx] < best_fitness:
                best_fitness = pop_fitness[min_idx]
                best_solution = population[min_idx].copy()

            offspring = []
            # produce offspring until fill (random parent selection as in paper)
            while len(offspring) < self.pop_size:
                p1 = random.choice(population)
                p2 = random.choice(population)
                if p1 == p2:
                    p2 = random.choice(population)
                c1, c2 = self.mEAX_crossover(p1, p2)
                # local search VND (lightweight)
                c1 = self.vnd_improvement(c1, max_iters=30)
                c2 = self.vnd_improvement(c2, max_iters=30)
                offspring.append(c1)
                offspring.append(c2)

            # combine and select best pop_size
            combined = population + offspring
            # compute/lookup fitness for combined
            combined_fitness = [self.calculate_fitness(ind) for ind in combined]
            sorted_idx = np.argsort(combined_fitness)
            selected_idx = sorted_idx[:self.pop_size]
            population = [combined[i] for i in selected_idx]
            pop_fitness = [combined_fitness[i] for i in selected_idx]

            fitness_history.append(best_fitness)
            if gen % 1 == 0:
                print(f"Gen {gen}: Best max route length = {best_fitness:.4f}")

        # final metrics
        routes = self.decode_solution(best_solution)
        route_distances = [self._route_distance(r) for r in routes]
        total_distance = sum(route_distances)
        balance_metric = max(route_distances) - min(route_distances) if route_distances else 0.0
        return routes, total_distance, best_fitness, balance_metric, fitness_history


In [2]:
distance_df = pd.read_csv(r'..\data\HN_distance_matrix.csv', index_col=0)
distance_matrix = distance_df.values

# Kiểm tra sơ bộ
print("Số thành phố:", distance_matrix.shape[0])
print("Ví dụ khoảng cách [0][1]:", distance_matrix[0][1])


Số thành phố: 127
Ví dụ khoảng cách [0][1]: 21.48


In [3]:
def detect_convergence(generation_fitness, tolerance=1e-3, window=5):
    for i in range(len(generation_fitness) - window):
        window_values = generation_fitness[i:i+window]
        if max(window_values) - min(window_values) < tolerance:
            return i + window
    print(f"Không phát hiện hội tụ sớm — thuật toán chạy đủ {len(generation_fitness)} thế hệ.")
    return len(generation_fitness)


In [4]:

# Run experiments for different numbers of salesmen
for m in range(1, 4):
    print(f"\n=== MEAX - Số người bán hàng (m) = {m} ===")

    start_time = time.time()

    MEAX = meax_mtsp(distance_matrix, m=m, pop_size=10, generations=30)
    routes, total_distance, max_route, balance_metric, fitness_per_generation = MEAX.run()

    end_time = time.time()
    exec_time = end_time - start_time

    route_distances = []
    for route in routes:
        if len(route) > 0:
            full_route = [0] + route + [0]
            distance = sum(distance_matrix[full_route[i]][full_route[i+1]] 
                           for i in range(len(full_route)-1))
            route_distances.append(distance)
        else:
            route_distances.append(0)

    balance_metric = max(route_distances) - min(route_distances) if route_distances else 0

    # Convergence analysis
    converged_gen = detect_convergence(fitness_per_generation)
    convergence_speed = (fitness_per_generation[0] - fitness_per_generation[-1]) / converged_gen if converged_gen > 0 else 0

    # Print results - all required metrics
    print(f"Tổng quãng đường: {total_distance:.2f}")
    print(f"Chiều dài route dài nhất (Max route length): {max_route:.2f}")
    print(f"Chênh lệch giữa các route (Balance metric): {balance_metric:.2f}")
    print(f"Thời gian thực thi: {exec_time:.4f} giây")
    print(f"Số vòng lặp cần thiết để hội tụ: {converged_gen}")
    print(f"Tốc độ hội tụ: {convergence_speed:.4f} đơn vị/gen")

    # Display individual routes
    for i, route in enumerate(routes):
        if len(route) > 0:
            full_route = [0] + route + [0]
            print(f" - Tuyến {i+1} ({route_distances[i]:.2f}): {full_route}")
        else:
            print(f" - Tuyến {i+1} (0.00): [0]")

    # Plot fitness evolution
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_per_generation, marker='o', linestyle='-', color='red', markersize=3)
    plt.title(f"MEAX - Fitness qua các thế hệ (m = {m})")
    plt.xlabel("Thế hệ")
    plt.ylabel("Max route length (fitness)")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


=== MEAX - Số người bán hàng (m) = 1 ===


MemoryError: 

NSGA-II không phải là lựa chọn tối ưu cho bài toán mTSP.
1. Tính bất ổn định cao: Tất cả 3 biểu đồ đều cho thấy dao động mạnh và không có xu hướng hội tụ rõ ràng
2. Tốc độ hội tụ chậm: Sau 500 thế hệ vẫn chưa đạt được trạng thái ổn định
3. NSGA-II được thiết kế cho đa mục tiêu, nhưng mTSP có cấu trúc đặc biệt